# Transfer learning with deep convolutional neural network for liver steatosis assessment in ultrasound images
Reproduce Results of [Transfer learning with deep convolutional neural network for liver steatosis assessment in ultrasound images](https://pubmed.ncbi.nlm.nih.gov/30094778/)


> We used a pre-trained CNN to extract features based on B-mode images. Next, using the neural features, we employed the support vector machine (SVM) algorithm to classify images containing fatty liver. Aside of fatty liver classification, it is clinically relevant to quantify the grade of liver steatosis. For this task, we used the extracted features and the Lasso regression method. In both cases, liver biopsy results served as a reference. The performance of the pro- posed approach was compared with the GLCM methods.





In [60]:
import sys
sys.path.append('../src')

from utils.reduce import reduce_pca
from utils.split import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

from itertools import product
import pickle
import pandas as pd
import numpy as np
import mlflow
import matplotlib.pyplot as plt

In [61]:
mlflow.set_experiment('logistic_regression_experiment')

## Feature Reduction/Selection

#### Upload Scattering Features

In [62]:
with open('../data/03_features/scattering_features.pickle', 'rb') as handle:
    scatter_dict = pickle.load(handle)
    df_scattering = scatter_dict['df']
    scattering_params = {'J':scatter_dict['J'],
                         'M':scatter_dict['M'],
                         'N':scatter_dict['N']}

#### Apply PCA

Since sklearn is used for PCA, the dataset will be transformed into a panda dataframe.

In [63]:
pca_n_components = 50
df_scattering_10 = reduce_pca(data=df_scattering, n_components=pca_n_components)

# Cross Validation using SVM Classification

> Methods that exclude outliers were used to normalize the features. Patient-specific leave-one-out cross-validation (LOOCV) was applied to evaluate the classification. In each case, the test set consisted of10 images from the same patient and the training set contained 540 images from the remaining 54 patients. For each training set, fivefold cross-validation and grid search were applied to indicate the optimal SVM classifier hyperparameters and the best kernel. To address the problem of class imbalance, the SVM hyperparameter C of each class was adjusted inversely proportional to that class frequency in the training set. Label 1 indicated the image containing a fatty liver and label −1 otherwise. 


In [64]:
df_train, df_test = train_test_split(df_scattering_10)

/home/mila/g/gauthies/.conda/envs/ultra/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [65]:
# Set the parameters by cross-validation
param_penalty= ['l1', 'l2', 'elasticnet', None]
#param_solver = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
params_C = [1, 10, 100, 1000]
params = list(product(['regression'],param_C, ))

In [66]:
standardize = True

df_train_pid = df_train.pop('id')
df_train_y = df_train.pop('class')


In [67]:

search_metrics = {}

for param in params:
    # Do cross-validation
    with mlflow.start_run():
        mlflow.log_param('pca_n',pca_n_components)
        mlflow.log_params(scattering_params)
        group_kfold = GroupKFold(n_splits=5)
        metrics = []
        for train_index, valid_index in group_kfold.split(df_train, 
                                                          df_train_y, 
                                                          df_train_pid):
            X_train, X_valid = df_train.iloc[train_index], df_train.iloc[valid_index]
            y_train, y_valid = df_train_y.iloc[train_index], df_train_y.iloc[valid_index]

            if standardize:
                scaler = StandardScaler()
                X_train = scaler.fit_transform(X_train)
                X_valid = scaler.transform(X_valid)
            
            mlflow.log_param('C',param[1])
            mlflow.log_param('Penalty',param[0])
            
            model =  LogisticRegression(C=param[1], penalty= param[0])
            #model = SVC(gamma=param[1], C=param[2])
            model.fit(X_train, y_train)
            predictions = model.predict(X_valid)
            acc = accuracy_score(y_valid, predictions)

            metrics.append(acc)
        
        search_metrics[str(param)] = np.mean(metrics)
        
        mlflow.log_metric('accuracy',np.mean(metrics))
        

ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

# Run Mlflow to see results

`!mlflow ui`

Should launch something like this:



In [ ]:
# !mlflow ui

# Test Prediction

In [ ]:
# Set a new mlflow experiment
# Use the best hyperparameters to train a model on the whole training data
# Test and record results!